In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def load_evaluations(base_dir="evals"):
    evaluations = {}
    for model in os.listdir(base_dir):
        model_path = os.path.join(base_dir, model)
        if os.path.isdir(model_path):
            evaluations[model] = {}
            for file in os.listdir(model_path):
                if file.endswith('_evaluation.npy'):
                    xai_method = file.split('_')[0]
                    file_path = os.path.join(model_path, file)
                    evaluations[model][xai_method] = np.load(file_path)
    return evaluations

# Load all evaluations
all_evaluations = load_evaluations()

# Compute averages and standard deviations
def compute_stats(eval_array):
    return np.mean(eval_array, axis=0), np.std(eval_array, axis=0)

stats = {model: {xai: compute_stats(eval_array)
                 for xai, eval_array in model_evals.items()}
         for model, model_evals in all_evaluations.items()}

In [3]:
np.set_printoptions(suppress=True, precision=4)

# Sanity / correctness check

In [4]:
def has_nan(arr):
    return np.isnan(arr).any()

for model in all_evaluations:
    for xai_method in all_evaluations[model]:
        # Verify no score is NaN
        if has_nan(all_evaluations[model][xai_method]):
            print(f"Model {model} with XAI method {xai_method} has NaNs")
        # check if all values are in the range [-1, 1]
        if (all_evaluations[model][xai_method] > 1).any() or (all_evaluations[model][xai_method] < -1).any():
            print(f"Model {model} with XAI method {xai_method} has values outside of [-1, 1]")

# Evaluation

In [5]:
all_evaluations["scibert"]["shap-partition"].shape

(271, 17, 3)

In [ ]:
def get_predictions(evaluations, model, method):
    shap_evals = evaluations[model][method]
    first_scores = shap_evals[:, :, 0]
    predictions = np.argmax(first_scores, axis=1)
    return predictions

In [30]:
llama3_preds=[3,6,6,5,3,8,5,14,11,8,1,11,0,3,16,8,2,13,0,4,3,2,0,0,15,15,4,14,0,4,9,11,15,15,13,11,3,10,10,13,5,0,1,3,0,10,10,1,2,14,0,8,7,0,0,1,12,4,10,11,2,15,12,6,0,13,11,12,7,0,7,1,3,7,6,5,4,5,4,3,14,3,5,12,13,14,14,12,7,12,2,10,14,14,10,15,9,11,3,9,2,2,7,0,13,15,9,10,5,4,13,12,8,2,10,0,15,11,14,1,8,7,2,7,5,3,3,6,7,14,1,12,10,14,4,15,0,6,13,14,2,10,9,12,2,1,4,7,7,5,5,8,8,13,5,13,15,7,4,2,2,6,7,8,13,6,15,13,4,1,12,2,9,6,4,1,9,12,5,15,6,5,6,3,7,15,7,15,2,8,9,2,5,7,1,10,3,4,13,10,6,0,5,11,9,9,2,11,8,13,14,8,15,12,0,14,9,13,4,10,12,2,8,12,12,1,15,1,13,11,13,8,10,6,15,14,8,9,9,6,4,12,11,5,14,10,12,10,0,15,8,7,7,11,1,4,8,2,2,10,1,15,6,3,3,1,3,0,4,5,12]
scibert_preds=[3,2,6,5,3,8,5,14,15,8,1,11,0,3,14,8,2,13,7,4,0,2,0,0,15,15,7,14,0,4,9,11,15,9,13,11,3,10,10,13,5,0,1,3,0,10,10,1,4,14,9,8,7,2,0,1,11,4,10,11,1,15,12,6,0,13,11,6,9,1,7,1,3,13,6,5,4,5,15,3,5,3,5,12,13,14,14,12,7,6,2,10,14,14,6,15,9,8,3,2,2,4,6,0,13,15,11,10,5,4,13,12,8,2,10,0,15,11,14,1,8,7,2,7,5,3,3,6,7,14,1,12,10,5,2,15,0,6,13,14,2,10,3,14,13,1,4,9,7,5,5,8,3,13,5,13,15,7,4,4,2,6,7,2,13,6,9,13,4,1,12,2,0,6,0,1,9,12,5,15,12,5,6,3,7,15,7,15,2,2,9,1,5,7,1,10,3,4,13,10,9,10,5,6,9,15,2,6,8,13,14,8,15,12,0,8,9,12,4,9,6,2,15,11,12,1,15,1,13,11,13,8,10,6,8,1,8,9,0,6,4,12,11,5,14,10,12,10,1,15,12,8,9,15,1,7,8,2,7,10,1,15,6,3,3,1,3,0,1,5,12]
unllama3_preds=[3,2,6,5,3,8,5,14,14,8,4,11,0,3,14,8,2,13,0,4,3,2,0,0,15,15,2,12,0,4,9,11,15,9,13,11,3,10,10,13,5,0,1,3,0,10,10,1,15,14,0,8,7,2,0,1,11,4,10,11,2,15,12,6,0,13,11,6,9,0,7,1,3,14,6,5,4,5,4,3,12,3,5,12,13,14,14,12,2,12,2,10,14,14,10,15,9,11,3,9,2,9,7,0,13,15,11,10,5,4,13,12,8,2,6,0,15,11,14,1,11,3,2,7,5,3,3,6,7,14,1,12,10,12,4,15,0,6,13,14,2,10,9,14,2,1,4,7,7,5,5,8,8,13,5,13,15,9,4,5,2,11,7,2,13,6,11,13,4,1,12,15,9,6,4,1,7,12,5,15,12,5,6,3,7,15,7,15,2,2,9,2,5,7,1,10,3,4,13,10,8,0,5,11,9,4,2,6,8,13,14,8,15,6,0,8,9,13,4,10,12,2,8,8,12,1,9,1,13,11,13,8,10,6,15,1,8,9,0,6,4,12,11,5,14,10,12,10,1,15,8,8,7,15,1,0,8,15,7,10,1,9,6,11,3,1,3,0,1,5,14]

In [49]:
# check if predictions are matching but they are numpy arrays
#assert np.array_equal(llama3_preds, get_predictions(all_evaluations, "llama3", "shap-partition"))

for i, pred in enumerate(get_predictions(all_evaluations, "unllama3", "attnlrp")):
    if unllama3_preds[i] != pred:
        print(f"i={i}, predicted_label={unllama3_preds[i]}, highest scoring label={pred}")
        print(all_evaluations["llama3"]["attnlrp"][i, :, 0])


i=8, predicted_label=14, highest scoring label=15
[-0.0003 -0.0002 -0.1582 -0.162  -0.2492 -0.0034 -0.0004 -0.0012  0.0007
 -0.0042 -0.0674  0.2874 -0.0005 -0.2856 -0.0031  0.6037  0.0027]
i=9, predicted_label=8, highest scoring label=7
[-0.028  -0.0074 -0.0001 -0.0242 -0.0016 -0.001  -0.0004  0.0334  0.7483
  0.0013  0.0018 -0.0007  0.0001 -0.0001 -0.0001 -0.0047 -0.    ]
i=27, predicted_label=12, highest scoring label=14
[-0.     -0.1235 -0.0042 -0.0018 -0.     -0.0025 -0.001  -0.0002 -0.0014
 -0.0007 -0.0317 -0.1342  0.5235 -0.0001  0.3686 -0.0001 -0.0001]
i=33, predicted_label=9, highest scoring label=7
[ 0.0001 -0.0046 -0.0001 -0.0443  0.0013 -0.     -0.0001  0.0685 -0.0021
 -0.0846 -0.0706 -0.0001  0.0001 -0.0009  0.      0.7859  0.0021]
i=34, predicted_label=13, highest scoring label=5
[-0.     -0.0001 -0.0165 -0.0001 -0.0002 -0.     -0.     -0.0006 -0.0272
 -0.     -0.0001 -0.0003 -0.      0.4322 -0.     -0.0037 -0.    ]
i=38, predicted_label=10, highest scoring label=15
[-0.00

In [33]:
def get_average_scores(evaluations, predictions, model, method):
    shap_evals = evaluations[model][method]

    # Create a mask to select scores for predicted classes
    mask = np.zeros_like(shap_evals, dtype=bool)
    mask[np.arange(len(predictions)), predictions] = True

    # Use the mask to select scores for predicted classes
    selected_scores = shap_evals[mask].reshape(shap_evals.shape[0], shap_evals.shape[2])

    # Calculate the average scores
    average_scores = np.mean(selected_scores, axis=0)

    return average_scores


def print_scores(model, method, preds):
    # Assuming all_evaluations is already loaded
    average_scores = get_average_scores(all_evaluations, preds, model, method)
    print(f"Average faithfulness scores ('{model}', '{method}'): {average_scores}")

print_scores("scibert", "attnlrp", scibert_preds)
print_scores("scibert", "shap-partition", scibert_preds)
print_scores("scibert", "shap-partition-tfidf", scibert_preds)
print_scores("scibert", "cplrp", scibert_preds)
print_scores("scibert", "lime", scibert_preds)
print_scores("scibert", "gradientxinput", scibert_preds)
print_scores("scibert", "integrated-gradient", scibert_preds)
print()
print_scores("llama3", "attnlrp", llama3_preds)
print_scores("llama3", "shap-partition", llama3_preds)
print_scores("llama3", "shap-partition-tfidf", llama3_preds)
print_scores("llama3", "cplrp", llama3_preds)
print_scores("llama3", "lime", llama3_preds)
print_scores("llama3", "gradientxinput", llama3_preds)
print_scores("llama3", "integrated-gradient", llama3_preds)
print()
print_scores("unllama3", "attnlrp", unllama3_preds)

Average faithfulness scores ('scibert', 'attnlrp'): [ 0.5495 -0.0612  0.213 ]
Average faithfulness scores ('scibert', 'shap-partition'): [ 0.3728 -0.0307  0.1229]
Average faithfulness scores ('scibert', 'shap-partition-tfidf'): [ 0.4455 -0.0263  0.1381]
Average faithfulness scores ('scibert', 'cplrp'): [0.481  0.0388 0.0623]
Average faithfulness scores ('scibert', 'lime'): [ 0.0738  0.2321 -0.0066]
Average faithfulness scores ('scibert', 'gradientxinput'): [ 0.1411  0.2402 -0.0237]
Average faithfulness scores ('scibert', 'integrated-gradient'): [ 0.5181 -0.0154  0.102 ]

Average faithfulness scores ('llama3', 'attnlrp'): [0.6321 0.0643 0.0821]
Average faithfulness scores ('llama3', 'shap-partition'): [0.5489 0.1911 0.0233]
Average faithfulness scores ('llama3', 'shap-partition-tfidf'): [0.4483 0.2698 0.0295]
Average faithfulness scores ('llama3', 'cplrp'): [0.6013 0.0995 0.0404]
Average faithfulness scores ('llama3', 'lime'): [ 0.019   0.257  -0.0029]
Average faithfulness scores ('llam

In [7]:
all_evaluations["llama3"]["attnlrp"][36, 3, 1]

0.12377475202083588

In [6]:
all_evaluations["unllama3"]["attnlrp"][71][:, 0]

array([ 0.0003,  0.7505, -0.0005, -0.1261, -0.0006, -0.0007,  0.    ,
        0.0001, -0.0003, -0.0003, -0.0923, -0.2396, -0.    , -0.006 ,
       -0.0123, -0.0039, -0.0061])